![imagenes](logo.png)

# Redes convolucionales

En una red neuronal estándar (densa), las capas se conectan bajo la suposición de que las entradas forman un vector sin estructura, todas las neuronas de una capa influyen en todas las de la siguiente y el índice de una neurona no tiene significado geométrico.

Este supuesto es adecuado para datos tabulares como los que hemos trabajado, pero falla cuando los datos presentan:

- estructura local,

- orden espacial o temporal,

- patrones que se repiten en distintas posiciones.

Las redes convolucionales surgen para modelar este tipo de datos.

Una CNN está diseñada para datos cuya representación natural es un *tensor indexado*. Es decir, ya no vemos al cliente como una serie de características (hora de llegada, sexo, estatura, actitud, ganancia mensual). En este caso tabular, si intercambiamos dos características de lugar (es decir, si intercambiamos dos columnas de lugar en la tabla) la información no se transforma, ni se pierde ni aumenta.

Pero en el caso de una CNN que cada **dato** (cliente) es un conjunto posible de diferentes características es un conjunto estructurado de características donde la posición sí importa. No estamos frente a una lista arbitraria de variables, sino ante un objeto geométrico: una imagen, un mapa, una señal bidimensional o tridimensional, es decir, un tensor cuyos índices codifican relaciones espaciales (o temporales).

Si en una imagen intercambiamos dos píxeles de lugar, la información sí cambia. El objeto representado puede deformarse, perder sentido o adquirir uno distinto. La vecindad, la continuidad y la localización contienen información semántica. Por ello, una CNN no solo aprende qué valores aparecen, sino dónde aparecen y cómo se relacionan localmente.




## Tipos de datos que analiza una CNN

Como dijimos, una CNN está diseñada para datos cuya representación natural es un tensor indexado. Es decir, no necesariamente se recibe una fila de una tabla, sino algo mucho más complejo: $$cliente\in\mathbb{R}^{I_1\times I_2\times...\times I_d}$$

Así, un cliente puede representarse como una fila, una tabla, un cubo o incluso un hipercubo de información. El caso $d=2$ corresponde a decir que cada cliente es *una tabla de información* y no una lista de características. Por ejemplo, si el antro es un hospital y el cliente es un paciente, entonces en CNN el cliente puede ser representado por una tabla de días x mediciones diarias por hora: filas = día del mes; columnas = mediciones horarias; y si además al paciente se le toma la presión o la glucosa entonces el cliente **una caja de información tridimensional**.

![imagenes](im06.png)


**Ejemplos:**

* imágenes → estructura espacial 2D,

* audio → estructura tiempo–frecuencia,

* texto → estructura local en secuencias,

* campos físicos → estructura espacial,

* mapas geográficos → estructura regional.

La idea central es que una CNN sustituye la conectividad global de las capas densas por operadores locales que se aplican repetidamente sobre la estructura del dato.


## Kernels y Filtros

Un **kernel** es simplemente una caja en tres dimensiones. Si su altura es 1, estamos hablando de una matriz. Además, su base es cuadrada.

A su vez, un filtro es un conjunto de kernels, todos de la misma altura y de la misma base, junto con un conjunto de sesgos. Esto es, $$\mathcal{F}=\{(K^{(1)},b^{(1)}),(K^{(2)},b^{(2)}),...,(K^{(r)},b^{(r)})\}$$ donde $K^{(j)}$ es un kernel y $b^{(j)}$ es un número.

## Operador de convolución

Sea $A$ una caja tridimensional y $K$ un kernel de su misma altura. La convolución de $K$ sobre $A$ se define como una operación entre ambos que siempre da como resultado una matriz. Esta operación está formada por un recorrido y una serie de cuentas que se hacen en cada paso del recorrido.

* **Si la altura de $A$ y $K$ es 1** (es decir, ambas son matrices)

![imagenes](im07.png)

El recorrido es el siguiente:


![imagenes](im08.png)

Los cálculos que se realizan en cada paso son una *multiplicación de Hadamard* y la suma de las entradas resultantes.

![imagenes](im09.png)

Por lo tanto en el paso $(p,q)$ se obtiene un número. Así, cuando se realiza el recorrido completo (es decir variamos $p$ y $q$ en todos sus posibles valores) obtenemos una matriz formada por los resultados del proceso anterior. Esta matriz se denomina **mapa de activación asociado al kernel $K$** o **canal**.

* **Si la altura de $A$ y $K$ es $>1$** (es decir, ambas son edificios de más de un piso). En este caso, el proceso es análogo al casi de altura 1: hacer el recorrido y en cada paso calcular la multiplicación de Hadamard y la suma de los elementos. Nuevamente, el resultado es una matriz (pues tanto $A$ como $K$ tienen la misma altura y por lo tanto solo hay movimiento en dos dimesiones):

![imagenes](im10.png)

Como quiera que sea, sin importar si $A$ y $K$ son de uno o mas pisos, notemos algo importante:
* Ambos tienen siempre la misma altura
* El resultado de la convolución siempre es una matriz. Es decir, una caja de altura 1
* Dicha matriz tiene base menor que la base de $A$: si $A$ tiene base $d_1\times d_2$ y $K$ tiene base $k\times k$ entonces el edificio resultante tiene base $d_1-k+1\times d_2-k+1$

De esta manera, si tomamos un filtro $\mathcal{F}$ con $r$ kernels y convolucionamos a $A$ con cada uno de ellos, **se obtiene un edificio de altura $r$**

## Capas convolucionales en una red neuronal

Tomemos la capa $i$ y la capa $i+1$; Diremos que la capa $i+1$ es convolucional si existe un filtro $\mathcal{F}$ formado por $r_{i+1}$ kernels y sesgos $$(K_{i+1}^{(1)},b_{i+1}^{(1)})\,,(K_{i+1}^{(2)},b_{i+1}^{(2)})\,,...\,,(K_{i+1}^{(r_{i+1})},b_{i+1}^{(r_{i+1})})$$

Digamos que la capa $i$ por alguna razón envía un edificio $A_i$ (de altura determinada) a la capa $i+1$. Entonces los kernels de $\mathcal{F}$ son todos de la misma altura que $A_i$. Esta capa recibe el edificio y le pasa todos los kernels seguido de los sesgos, de modo que obtendrá un edificio $Z_{i+1}$ de altura $r_{i+1}$.

Cada uno de los pisos es una matriz. De esta manera, el pensamiento crudo es $$z_{i+1}^{(s,p,q)}=\mbox{ valor que se encuentra en la posición }p,q\mbox{ del piso }s$$

Luego, se aplica la función de activación $f_{i+1}$, que generalmente es una ReLU, a todos los elementos de $Z_{i+1}$ lo que produce un edificio $A_{i+1}$ de pensamientos procesados y altura $r_{i+1}$: $$A_{i+1}=f_{i+1}(Z_{i+1})$$

## Padding

Hasta ahora, cuando un kernel se desplaza sobre un edificio, solo permitimos superposiciones completas. Este esquema se conoce como **convolución válida** (valid convolution) y tiene una consecuencia inevitable: cada vez que aplicamos un filtro, la base del edificio que se recibe disminuye. 

En concreto: la capa $i$ produce un edificio $A_i$, que puede tener altura 1, de base $d_1\times d_2$. La capa $i+1$ lo recibe y aplica un filtro. Digamos que cada kernel tiene base $k\times k$ (y altura igual a la altura de $A_i$). Entonces $Z_{i+1}$ tiene base $d_1-k+1\times d_2-k+1$

El padding consiste en extender artificialmente cada piso del edificio $A_i$ antes de aplicar la convolución. Esta extensión se hace agregando filas y columnas alrededor del edificio original, usualmente llenas con ceros.

Es decir, se toma $A_i$ y a cada piso se le agrega un borde de ancho $p$. Por lo tanto al pasar por la capa $i+1$ obtenemos $Z_{i+1}$ que tendrá base $$d_1+2p-k+1\times d_2+2p-k+1$$

En particular si $k$ es impar, podemos tomar $p=\frac{k-1}{2}$ y por lo tanto $Z_i$ tiene base de tamaño $d_1\times d_2$. Esto es: se conserva el tamaño de $A_i$.

## Pooling

Hasta ahora, en las capas convolucionales, el kernel recorre el edificio y produce nuevos valores. El **pooling** es distinto: **No crea nuevos pensamientos; resume los que ya existen.**

El pooling es una operación que se aplica después de la convolución y la activación, y actúa de manera independiente sobre cada piso del edificio. Se trata de un barrendero que en cada piso, junta lo más relevante de cada zona y deja menos información, pero más concentrada.

Supongamos que $A_{i+1}$ es la salida de la capa $i+1$, con altura $r_{i+1}$ y base $d_1\times d_2$. Recordemos que los pisos son matrices. El pooling divide cada piso en rectángulos de tamaño $k\times l$ y reemplaza cada valor por un solo valor. El resultado es un nuevo edificio $P_{i+1}$ de la misma altura $r_{i+1}$ pero con pisos de menor tamaño.

### Tipos de pooling

**Max pool**: en cada región $k\times l$ se toma al valor $$p_{i+1}^{(s,u,v)}=\max\{a_{i+1}^{(s,p,q)}:(p,q)\in\mbox{región }(u,v)\}$$

**Average pooling:** en cada región se toma el promedio $$p_{i+1}^{(s,u,v)}=\frac{1}{kl}\sum_{(p,q)\in\mbox{región }(u,v)}a_{i+1}^{(s,u,v)}$$

## Estructura típica de una red neuronal con convolución

En resumen, el flujo de información hacia adelante a través de una capa de convolución es

$$A_i \xrightarrow{\text{padding}}\tilde{A}_{i}\xrightarrow{\text{convolución + sesgo}}Z_{i+1}\xrightarrow{f_{i+1}}A_{i+1}\xrightarrow{\text{pooling}} P_{i+1}$$

De esta manera, la estructura típica de una red neuronal con convolución es $$\mbox{Entrada}\xrightarrow{}(\mbox{Conv+Act+Pool})^{\times k}\xrightarrow{}\mbox{Flatten}\xrightarrow{}\mbox{Capas densas}\xrightarrow{}\mbox{Salida}$$

![imagenes](im11.png)

## Preprocesamiento

Recordemos que los datos con los que trabajamos **no son renglones de una tabla**, sino que son edificios de información (posiblemente de 1 piso).

De esta manera, si tenemos los datos $$\mathcal{A}=\{A_1,A_2,...,A_{d}\}$$ entonces el procesamiento debe hacerse ligeramente diferente.

Lo primero: todos los elementos de $\mathcal{A}$ deben ser edificios con las mismas dimensiones: tamaño de la base y altura.

Una vez que todos los clientes han sido llevados a edificios con la misma forma, el siguiente paso es normalizar la información.

En una CNN, el reescalado no se hace pensando en columnas, como en el caso tabular, sino piso por piso y celda por celda. Cada valor del edificio representa una intensidad, una medición o una magnitud física localizada, por lo que es indispensable que todos los edificios vivan en la misma escala numérica.

Por ejemplo, en imágenes es común dividir todos los valores entre 255 para llevarlos al intervalo $[0,1]$. En otros casos, puede utilizarse una normalización por media y desviación estándar, aplicada globalmente al conjunto de edificios o canal por canal.

Formalmente, si $A_i^{(s,p,q)}$ representa el valor en el piso $s$ y posición $(p,q)$ del edificio $A_i$, el reescalado consiste en aplicar una transformación del tipo

$$\tilde{A}_i^{(s,p,q)}=\frac{A_i^{(s,p,q)}-\mu_s}{\sigma_s}$$ 

donde $\mu_s$ y $\sigma_s$ son parámetros asociados al piso $s$ (canal $s$). En muchos problemas prácticos, estos valores son fijos y conocidos de antemano; en otros, se estiman a partir del conjunto de entrenamiento.

